In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [3]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [5]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [21]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [6]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [9]:
 def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    # convert to lowercase
    data["tweet"] = data["tweet"].str.lower()
    # remove URLs
    data["tweet"] = data["tweet"].apply(
        lambda x: re.sub(r'https?://\S+|www\.\S+', '', x)
    )
    # remove punctuations
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    # remove digits
    data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)
    # remove stopwords
    data["tweet"] = data["tweet"].apply(
        lambda x: " ".join(word for word in x.split() if word not in sw)
    )
    # stemming
    data["tweet"] = data["tweet"].apply(
        lambda x: " ".join(ps.stem(word) for word in x.split())
    )
    return data["tweet"]


In [13]:
txt = 'great product. i like it'

In [14]:
txt

'great product. i like it'

In [15]:
preprocessed_txt = preprocessing(txt)

In [16]:
preprocessed_txt

0    great product like
Name: tweet, dtype: object

In [17]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [20]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [28]:
txt = "grea awesome product"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'